In [1]:
import pickle

from dotenv import load_dotenv
import numpy as np
import os
load_dotenv()

True

In [2]:
import re
import httpx
from openai import OpenAI
client = OpenAI()
class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(model="gpt-4o-mini", messages=self.messages)
        return completion.choices[0].message.content


In [3]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

In [4]:
action_re = re.compile('^Action: (\w+): (.*)')

def wikipedia(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]


def simon_blog_search(q):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": q,
    })
    return response.json()[0]["text"]

def calculate(what):
    print("calculation started")
    return eval(what)


known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [14]:
bot = ChatBot(prompt)

In [15]:
next_prompt = "how many neighbours does India have"
result = bot(next_prompt)

In [ ]:
print(result)

In [ ]:
bot.messages

In [17]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [ ]:
result.split('\n')

In [ ]:
actions[0]

In [ ]:
actions[0].groups()

In [ ]:
if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"

In [29]:
result = bot(next_prompt)

In [ ]:
print(result)

In [ ]:
bot.messages

In [ ]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
else:
    print(result)

In [5]:
def query(question, max_turns=5):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [7]:
print(query("what is five multipled by 37"))

Thought: I need to calculate the product of five and thirty-seven to find the answer.
Action: calculate: 5 * 37
PAUSE
 -- running calculate 5 * 37
calculation started
Observation: 185
Answer: Five multiplied by thirty-seven is 185.
Answer: Five multiplied by thirty-seven is 185.


In [ ]:
print(query("What does England share borders with?"))

In [ ]:
print(query("Fifteen * twenty five"))

In [ ]:
print(query("Has Simon been to Madagascar?"))